In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [ ]:
info1 = json.loads(open('runs/1/info.json').read())

In [12]:
p = figure(title='Confusion Matrix', plot_width=600, plot_height=400)

xy = np.array(info1['prop_acc_uncertainty_std'])
area = np.trapz(y=xy[:, 1], x=xy[:, 0])
print('std-dropout', area)
p.line(xy[:, 0], xy[:, 1], legend='std-dropout: {0:.6f}'.format(area), color='blue', line_width=3)

xy = np.array(info1['prop_acc_uncertainty_entropy_proba'])
area = np.trapz(y=xy[:, 1], x=xy[:, 0])
print('entropy-dropout', area)
p.line(xy[:, 0], xy[:, 1], legend='entropy-dropout: {0:.6f}'.format(area), color='red', line_width=3)

xy = np.array(info1['prop_acc_uncertainty_entropy'])
area = np.trapz(y=xy[:, 1], x=xy[:, 0])
print('entropy-ml', area)
p.line(xy[:, 0], xy[:, 1], legend='entropy-ml: {0:.6f}'.format(area), color='green', line_width=3)

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

std-dropout 0.993576440876
entropy-dropout 0.993632493253
entropy-ml 0.993663445791
